# Download de PDFs da Melbourne Punch

Este notebook demonstra como fazer download de PDFs de edições da Melbourne Punch no Trove. No entanto, abordaremos um tipo de download baseado em URLs em um arquivo \`records.csv\`.

Esse arquivo pode ser obtivo através da ferramenta de bulk download do Trove.

---

Vamos começar importando as bibliotecas necessárias e configurando nosso ambiente.

In [ ]:
import os                 
import traceback
from typing import (
    List,
    Dict,
    Tuple,
    Optional
)
import concurrent.futures     
from datetime import datetime 
import re                    
import pandas as pd         
import requests      
from tqdm.notebook import tqdm

### 1. Configurações Iniciais e Variáveis Globais

Esta seção define as principais variáveis que controlam o comportamento do script de download.

Você pode ajustar esses valores para personalizar a operação de acordo com suas necessidades.

In [ ]:
CSV_FILE: str = 'newRecords.csv'
PDF_OUTPUT_DIR: str = 'PDFs'
NUM_DOWNLOAD_THREADS: int = 10

In [ ]:
if not os.path.exists(PDF_OUTPUT_DIR):
    os.makedirs(PDF_OUTPUT_DIR)
    print(f"Diretório '{PDF_OUTPUT_DIR}' criado com sucesso.")
else:
    print(f"Diretório '{PDF_OUTPUT_DIR}' já existe.")

### 2. Carregamento dos Dados do Arquivo CSV

Esta célula carrega os dados do arquivo CSV especificado na variável `CSV_FILE`
e exibe uma mensagem de sucesso ou erro, dependendo do resultado.

In [ ]:
try:
    df: pd.DataFrame = pd.read_csv(CSV_FILE)
    print(f"Arquivo '{CSV_FILE}' carregado com sucesso. {len(df)} registros encontrados.")

except FileNotFoundError:
    print(f"ERRO: Arquivo CSV não encontrado em '{CSV_FILE}'.")
    print("Por favor, verifique o caminho e execute o notebook anterior se necessário.")
    raise FileNotFoundError(f"Arquivo CSV não encontrado em '{CSV_FILE}'")
except Exception as e:
    print(f"ERRO CRÍTICO: Erro inesperado ao carregar o CSV '{CSV_FILE}'.")
    print(f"Detalhes: {e}")
    raise

### 3. Função para Download de PDFs Individuais

Essa função, `download_pdf`, foi projetada para recuperar um único documento digital (especificamente, um arquivo PDF) de seu local on-line e salvá-lo em seu armazenamento local.

Veja como ela funciona:

1.  **Verificação de cópias existentes:** A função primeiro verifica se o documento já foi baixado e salvo. Se houver uma cópia com o mesmo nome no local especificado, a função pulará o download e informará que o arquivo existente será usado. Isso evita downloads desnecessários.

2.  **Baixar o documento:** Se for necessário fazer o download do documento, nós o buscamos em seu endereço da Web. Ele também verifica se há erros durante esse processo, como o fato de o documento não ter sido encontrado no endereço fornecido.

3.  **Acompanhamento do progresso do download:** Para documentos maiores, a função exibe uma barra de progresso. Esse indicador visual mostra quanto do documento foi recuperado com êxito.

4.  **Salvando o documento:** Depois que o documento é totalmente baixado, a função o salva no local especificado no armazenamento do computador.

5.  **Relatório do resultado:** Por fim, a função informa o resultado da operação. Ela indica se o download foi bem-sucedido, ignorado (porque o arquivo já existia) ou se ocorreu um erro. Se tiver ocorrido um erro, a função fornecerá detalhes sobre a natureza do problema.

In [ ]:
def download_pdf(unique_download_id: str, pdf_url: str, output_path: str) -> Dict:
    if os.path.exists(output_path):
        return {'unique_download_id': unique_download_id, 'status': 'skipped', 'filename': output_path, 'reason': 'already exists'}

    try:
        response = requests.get(pdf_url, stream=True, timeout=60)
        response.raise_for_status()

        total_size = int(response.headers.get('content-length', 0))
        chunk_size = 8192

        progress = tqdm(total=total_size, unit='iB', unit_scale=True,
                        desc=f'Baixando {unique_download_id}', leave=False)

        with open(output_path, 'wb') as pdf_file:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:
                    progress.update(len(chunk))
                    pdf_file.write(chunk)
                    
        progress.close()

        return {'unique_download_id': unique_download_id, 'status': 'success', 'filename': output_path}

    except requests.exceptions.RequestException as e:
        return {'unique_download_id': unique_download_id, 'status': 'failed', 'error': f"Erro na requisição: {e}"}
    except Exception as e:
        return {'unique_download_id': unique_download_id, 'status': 'failed', 'error': f"Erro inesperado: {e}", 'traceback': traceback.format_exc()}

## 4: Orquestrando a recuperação em massa de documentos digitais

Por fim, recuperaremos vários documentos digitais (arquivos PDF) com base na lista preparada anteriormente.

Veja a seguir um detalhamento das etapas:

1.  **Preparação da lista de download:** Primeiro, o sistema cria uma lista detalhada de todos os documentos que precisam ser recuperados. Para cada documento, essa lista inclui seu identificador exclusivo, seu endereço da Web (URL) e o local onde ele deve ser salvo em seu computador.

2.  **Monitoramento do progresso geral:** Uma barra de progresso é exibida para fornecer uma representação visual de quantos documentos foram recuperados com êxito do número total de documentos na lista. Isso permite que você acompanhe a conclusão geral do processo de recuperação.

3.  **Registro dos documentos recuperados:** À medida que cada documento é baixado com êxito (ou se foi ignorado porque já existia), o sistema mantém um registro do identificador do documento e do local salvo no seu computador.

4.  **Tratamento de problemas:** Se ocorrer algum problema durante a recuperação de um documento específico (por exemplo, o endereço da Web está incorreto ou há um problema de rede), o sistema tentará relatar esses erros.

5.  **Finalização do processo:** Depois que todos os documentos da lista tiverem sido processados (baixados, ignorados ou resultaram em erro), uma mensagem indicará que todo o processo de recuperação foi concluído. O sistema também o lembrará da pasta onde todos os documentos baixados com êxito podem ser encontrados.

In [ ]:
url_to_filepath_map: Dict[str, str] = {}
pdf_downloads_unique: List[Tuple[str, str, str]] = []
item_id_to_filepaths: Dict[str, List[str]] = {}

print("\nPreparando lista de PDFs para download")

PAGE_ID_PATTERN = re.compile(r'nla\.news-page(\d+)')

for row_idx, row in tqdm(df.iterrows(), total=len(df), desc="Mapeando URLs de PDF"):
    original_item_id: str = str(row['id']) 
    pdf_url_raw: Optional[str] = row.get('trove_pdf_url')

    if pd.notna(pdf_url_raw):
        potential_urls: List[str] = str(pdf_url_raw).split('|')

        for url_idx, url in enumerate(potential_urls):
            cleaned_url = url.strip()
            
            if cleaned_url.startswith('http') and len(cleaned_url) > 10:
                pdf_filename_base = ""
                unique_download_id = ""
                
                match = PAGE_ID_PATTERN.search(cleaned_url)
                if match:
                    page_id = match.group(1)
                    pdf_filename_base = f"{page_id}.pdf"
                    unique_download_id = page_id
                else:
                    pdf_filename_base = f"{original_item_id}_{url_idx+1}.pdf"
                    unique_download_id = f"{original_item_id}_{url_idx+1}"
                    print(f"Formato de URL Trove inesperado para ID {original_item_id} (URL: '{cleaned_url}'). Usando nome de arquivo padrão: '{pdf_filename_base}'")

                output_path = os.path.join(PDF_OUTPUT_DIR, pdf_filename_base)
                
                if cleaned_url not in url_to_filepath_map:
                    url_to_filepath_map[cleaned_url] = output_path
                    pdf_downloads_unique.append((unique_download_id, cleaned_url, output_path))
                    print(f"Adicionado download único: {pdf_filename_base} da URL: {cleaned_url}")
                else:
                    print(f"URL '{cleaned_url}' já enfileirada para download como '{url_to_filepath_map[cleaned_url]}'.")
                
                if original_item_id not in item_id_to_filepaths:
                    item_id_to_filepaths[original_item_id] = []

                item_id_to_filepaths[original_item_id].append(output_path)

            else:
                print(f"URL de PDF inválida ou malformada detectada para ID original {original_item_id} (URL {url_idx+1}): '{url}'. Ignorando.")
    else:
        print(f"PDF URL ausente ou NaN para ID original {original_item_id}. Nenhuma URL encontrada para download.")

print(f"\nTotal de {len(pdf_downloads_unique)} PDFs ÚNICOS elegíveis para download (após desduplicação de URLs).")
print("-" * 50)

print(f"Finalizada preparação. Total de {len(pdf_downloads_unique)} PDFs únicos para processar.")

downloaded_files_status: Dict[str, str] = {}
failed_downloads_list: List[Dict] = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_DOWNLOAD_THREADS) as executor:
    futures = {
        executor.submit(download_pdf, unique_id, pdf_url, output_path): unique_id
        for unique_id, pdf_url, output_path in pdf_downloads_unique
    }
    
    for future in tqdm(concurrent.futures.as_completed(futures),
                       total=len(futures),
                       desc="Progresso Geral do Download de PDFs"):
        unique_id = futures[future]
        try:
            result: Dict = future.result()  

            if result['status'] == 'success' or result['status'] == 'skipped':
                downloaded_files_status[result['unique_download_id']] = result['filename']
            else: 
                failed_downloads_list.append(result)

        except Exception as e:
            print(f"Erro inesperado ao processar resultado para ID de download {unique_id}: {e}\n{traceback.format_exc()}")
            failed_downloads_list.append({'unique_download_id': unique_id, 'status': 'failed', 'error': f"Erro de processamento: {e}", 'traceback': traceback.format_exc()})

print("\n--- Processo de Download de PDFs Concluído ---")
print(f"Total de downloads únicos considerados: {len(pdf_downloads_unique)}")

successful_downloads_count = 0
skipped_downloads_count = 0

for future in futures:
    if future.done():
        try:
            result = future.result()
            if result['status'] == 'success':
                successful_downloads_count += 1
            elif result['status'] == 'skipped':
                skipped_downloads_count += 1
        except Exception as e:
            print(f"Erro ao obter resultado de um future para contagem final: {e}")


print(f"PDFs baixados com sucesso (novos): {successful_downloads_count}")
print(f"PDFs pulados (já existiam): {skipped_downloads_count}")
print(f"PDFs com falha no download: {len(failed_downloads_list)}")
print(f"Todos os PDFs baixados estão em: '{os.path.abspath(PDF_OUTPUT_DIR)}'")

if failed_downloads_list:
    print("\n--- DETALHES DOS DOWNLOADS FALHOS: ---")
    for failure in failed_downloads_list:
        print(f"  ID de Download: {failure.get('unique_download_id', 'N/A')}")
        print(f"  Erro: {failure.get('error', 'N/A')}")
        print("-" * 20)

print("\nAnálise completa. Você pode verificar a pasta de saída para os arquivos PDF.")

### 5. Atualizando o DataFrame com os Nomes dos Arquivos Baixados

Agora, vamos adicionar uma nova coluna ao nosso DataFrame original (`df`) que registrará os nomes dos arquivos PDF que foram baixados ou já existiam para cada entrada. 

Isso permite uma rastreabilidade fácil entre seus dados e os arquivos físicos.

In [ ]:
df['downloaded_pdf_filenames'] = df['id'].apply(lambda x: item_id_to_filepaths.get(str(x), []))

selected_columns = ['id', 'trove_page_url', 'date', 'page', 'illustrated', 'type', 'downloaded_pdf_filenames']

try:
    df_selected = df[selected_columns]
    new_csv_filename = 'selected_records_with_downloaded_pdfs.csv'
    df_selected.to_csv(new_csv_filename, index=False)
    
except KeyError as e:
    print(f"\nATENÇÃO: Uma das colunas selecionadas não foi encontrada no DataFrame original: {e}.")
    print("Por favor, verifique se todas as colunas existem antes de tentar selecioná-las.")
    raise